# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
import pytz
import datetime 
import cPickle as pickle
import astral
import astropy.time as aptime
from scipy import stats
from scipy import interpolate
from scipy import optimize
import os
import time
from matplotlib import gridspec

In [ ]:
from matplotlib import rc
from matplotlib import rcParams
rc('xtick', labelsize=15) 
rc('ytick', labelsize=15) 
rc('axes', labelsize=20) 
rc('axes', linewidth=2)

rcParams['axes.titlesize'] = 25
rcParams['legend.fontsize'] = 20
rcParams['patch.linewidth'] = 1
rcParams['axes.titlepad'] = 30

In [ ]:
pkl_file = open("GS-env-2014-19_p2.pkl", 'rb')
dictionary = pickle.load(pkl_file)
pkl_file.close()
dictionary_copy = copy.copy(dictionary)

# Merge sensors

In [ ]:
env_data = dictionary_copy

sensor_list = ['T_M1+Y','T_M1-Y','T_TrussAirLow+X','T_TrussAirMid+X','T_TrussAirTop+X',
               'T_TrussSurLow+X','T_TrussSurMid+X','T_TrussSurTop+X','T_twr']

n=0
for sensor in sensor_list:
    
    #  convert list to arrays and make an MJDOBS column
    env_data[sensor]['vals'] = np.array(env_data[sensor]['vals'])
    env_data[sensor]['dts'] = np.array(env_data[sensor]['dts'])
    env_data[sensor]['MJDOBS'] = aptime.Time(env_data[sensor]['dts'], format='datetime').mjd
    
    #  Get rid of zeros and outliers 
    cond1 = env_data[sensor]['vals'] != 0.
    cond2 = np.abs(env_data[sensor]['vals']) < 20.
    
    ind = np.where(cond1 & cond2)[0]
    
    env_data[sensor]['vals'] = env_data[sensor]['vals'][ind]
    env_data[sensor]['dts'] = env_data[sensor]['dts'][ind]
    env_data[sensor]['MJDOBS'] = env_data[sensor]['MJDOBS'][ind]
    
    #  merge other sensor data columns to M1+Y data table
    if n==0:
        df = pd.DataFrame(columns = ['dts','MJDOBS',sensor])
        df[sensor] = pd.Series(env_data[sensor]['vals'])
        df['dts'] = pd.Series(env_data[sensor]['dts'])
        df['MJDOBS'] = pd.Series(env_data[sensor]['MJDOBS'])
        df = df.sort_values(by = ['dts'])
        n=1
   
    else:
        newdf = pd.DataFrame(columns = [sensor,'dts'])
        newdf[sensor] = pd.Series(env_data[sensor]['vals'])
        newdf['dts'] = pd.Series(env_data[sensor]['dts'])
        newdf = newdf.sort_values(by = ['dts'])
        
        df = pd.merge_asof(df,newdf, on='dts',tolerance=pd.Timedelta('30min')) #before it was 30min
        
env_data = df
env_data['dts'] = env_data['dts'].dt.tz_localize(pytz.UTC)

date upper limit 

In [ ]:
env_data.keys()

In [ ]:
print len(env_data)
for sensor in sensor_list:
    print sum(~np.isfinite(env_data[sensor]))  

date lower bound 

In [ ]:
%matplotlib notebook

date = datetime.date(year = 2017, month = 8, day =3) #calibration date 
ind1 = np.where(env_data['dts'] == date)[0]
ind2 = np.where(env_data['dts'] > date)[0]

Checking right at the change date  

In [ ]:
%matplotlib notebook

date1 = datetime.datetime(year = 2017, month = 8, day =2)
date2 = datetime.datetime(year = 2017, month = 8, day =4)

cond1 = env_data['dts'] >= date1
cond2  = env_data['dts'] <= date2

ind = np.where(cond1 & cond2)[0]

cp = env_data.iloc[ind]

plt.plot(cp['dts'],cp['T_M1+Y'],'.')
plt.plot(cp['dts'],cp['T_M1-Y'],'.')
plt.xticks(rotation=30)

In [ ]:
#print af.loc[0,'T_M1+Y']
print bf.loc[len(bf)-1,'T_M1+Y'] 

#print af.loc[0,'T_M1-Y']
print bf.loc[len(bf)-1,'T_M1-Y'] 

#print af.loc[0,'dts']
print bf.loc[len(bf)-1,'dts'] 

# Calculating measurement offsets

There is seasonal variation in each sensor so we will fold them over and subtract the last six months of data

In [ ]:
''' plt.figure(1,figsize=[8,6])
    plt.plot(x,y,'.',color='silver',markersize=3)
    plt.plot(x2,y2,'.',color='b',markersize=3)
    plt.plot(x,func(x,*popt),'r',lw=3)
    plt.xticks([])
    plt.xlabel('Time')
    plt.ylabel('M+Y Temperature [$C^\circ$]')
    #plt.legend(['BF cal','AF cal', 'BF cal fit'],fontsize=15)
    plt.tick_params(axis='both',which = 'both',direction='in', width=2,length = 7, bottom = False, top=True,)
    plt.show()
    
    plt.savefig('/Users/melisaT/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/plots/SPIE_poster_figs/cal_fit.png')'''


# All years combined 

In [ ]:
def find_offset_yrs_combined(sensor):
    
    #  Split data into before and after calibration data frames. 
    date = datetime.date(year = 2017, month = 8, day =2)
    ind1 = np.where(env_data['dts'] <= date)[0]
    ind2 = np.where(env_data['dts'] > date)[0]

    bf = env_data.iloc[ind1].sort_values(by='dts').set_index(np.arange(len(ind1)))
    af = env_data.iloc[ind2].sort_values(by = 'dts').set_index(np.arange(len(ind2)))
    
    #  Replace year of dts to 2000
    bf.loc[:,'dts'] = bf['dts'].apply(lambda dt: dt.replace(year = 2000 )) 
    bf.loc[:,'MJDOBS'] = aptime.Time(bf['dts'].dt.to_pydatetime(), format='datetime').mjd  # Calculate MJD date after changing year 
    bf = bf.sort_values(by ='dts').set_index(np.arange(len(bf))).dropna(subset=[sensor])  #  Sort the values in order by date

    af.loc[:,'dts'] = af['dts'].apply(lambda dt: dt.replace(year = 2000 ))
    af.loc[:,'MJDOBS'] = aptime.Time(af['dts'].dt.to_pydatetime(), format='datetime').mjd
    af = af.sort_values(by ='dts').set_index(np.arange(len(af))).dropna(subset=[sensor])
    
    
    #  function that fits a curve to data before calibration   
    def func(x, a, b):
        return a*x + b
    
    #date = datetime.date(year = 2000, month = 8, day =2)
    date = datetime.date(year = 2000, month = 7, day =1)


    ind = np.where(bf['dts'] < date)[0]

    bf.loc[:,'dts'] = bf['dts'].apply(lambda dt: dt.replace(year = 2000 ))  #  Replace year of dts to 2000
    bf.loc[:,'MJDOBS'] = aptime.Time(bf['dts'].dt.to_pydatetime(), format='datetime').mjd  # Calculate MJD date after changing year 
    bf = bf.sort_values(by ='dts').set_index(np.arange(len(bf))).dropna(subset=[sensor])  #  Sort the values in order by date

    af.loc[:,'dts'] = af['dts'].apply(lambda dt: dt.replace(year = 2000 ))
    af.loc[:,'MJDOBS'] = aptime.Time(af['dts'].dt.to_pydatetime(), format='datetime').mjd
    af = af.sort_values(by ='dts').set_index(np.arange(len(af))).dropna(subset=[sensor])
    

#  Input arrays for curve fit
    x = bf.loc[ind,'MJDOBS']
    y = bf.loc[ind,sensor]
    
    x2 = af['MJDOBS']
    y2 = af[sensor]
     
    #  model calibration model
    popt,pcov = optimize.curve_fit(func,x,y)
    
    # minimize the error between the post calibration and the fit model of before calibration
    def J(offset):
        x = af['MJDOBS']  #  dates for fit
        y = af[sensor]  # temp post calibration 
        m = func(x, *popt) # fit of pre-calibrated temp
        return np.sum((y[None,:] - m[None,:] + offset[:,None])**2,axis=1) 

    l = np.linspace(-10,10,2000)
    Jarr = J(l)
    idx = np.argmin(Jarr) 
    return l[idx]    

In [ ]:
find_offset_yrs_combined('T_M1-Y')

# One year at a time

In [ ]:
def find_offset(sensor,yr):
    
    if yr > 2016:
        print('Data have already been recalibrated in 02/03/2017 so theoretically values shouls be accurate')
    
#  Split data into before and after calibration data frames. 
    date = datetime.date(year = 2017, month = 8, day =2) # last day before calibration
    ind1 = np.where(env_data['dts'] <= date)[0]
    ind2 = np.where(env_data['dts'] > date)[0]

    bf = env_data.iloc[ind1].sort_values(by='dts').set_index(np.arange(len(ind1)))
    af = env_data.iloc[ind2].sort_values(by = 'dts').set_index(np.arange(len(ind2)))
    
    # pick all points in specified year
    bf_date1 = datetime.datetime(year = yr, month = 1, day =1) # lower bound of data beofre calibration
    bf_date2 = datetime.datetime(year = yr, month = 12, day = 31)

    cond1 = bf['dts'] > bf_date1
    cond2  = bf['dts'] <= bf_date2
    bf_ind = np.where(cond1 & cond2)[0]
        
    bf = bf.iloc[bf_ind].sort_values(by='dts').set_index(np.arange(len(bf_ind))).dropna(subset=[sensor])
    
    #  Replace year of after to coincide with year of before
    af.loc[:,'dts'] = af['dts'].apply(lambda dt: dt.replace(year = yr))
    af.loc[:,'MJDOBS'] = aptime.Time(af['dts'].dt.to_pydatetime(), format='datetime').mjd
    af = af.sort_values(by ='dts').set_index(np.arange(len(af))).dropna(subset=[sensor])
    
    #  function that fits a curve to data before calibration   
    def func(x, a, b):
        return a*x + b

#  Input arrays for curve fit
    x = bf['MJDOBS']
    y = bf[sensor]
    
    x2 = af['MJDOBS']
    y2 = af[sensor]
     
    #  model calibration model
    popt,pcov = optimize.curve_fit(func,x,y)
    
    # minimize the error between the post calibration and the fit model of before calibration
    def J(offset):
        x = af['MJDOBS']  #  dates for fit
        y = af[sensor]  # temp post calibration 
        m = func(x, *popt) # fit of pre-calibrated temp
        return np.sum((y[None,:] - m[None,:] + offset[:,None])**2,axis=1) 

    l = np.linspace(-10,10,2000)
    Jarr = J(l)
    idx = np.argmin(Jarr) 
    return l[idx]    

In [ ]:
find_offset('T_M1+Y',2016)

# Summary of Results

Make an empty dataframe that will store values

In [ ]:
yr_list = [2014,2015,2016]
offset_summary = pd.DataFrame(columns=yr_list,index=sensor_list)
offset_summary['combined'] = np.nan

Fill values of dataframe

In [ ]:
for s in sensor_list:
    print s 
    offset_summary.loc[s,'combined'] = find_offset_yrs_combined(s)
    for yr in yr_list:
        print yr
        offset_summary.loc[s,yr] = find_offset(s,yr)

In [ ]:
print(offset_summary)

In [ ]:
offset_summary.loc[:,[2014,2015,2016]].mean(axis=1)

In [ ]:
offset_summary.loc[:,[2014,2015,2016]].std(axis=1)

# Figures

Apply sensor offsets and calculate difference across primary mirror. 

In [ ]:
cp = copy.deepcopy(env_data)

In [ ]:
%matplotlib notebook

date = datetime.date(year = 2017, month = 8, day =3)

#  AO system requirements
cond1 = (env_data['dts'] > date) 
ind = np.where(cond1)[0]

cond2 = (env_data['dts'] < date) 
ind2 = np.where(cond2)[0]

af = cp.iloc[ind]
bf = cp.iloc[ind2]

bf.loc[ind2,'T_M1+Y'] = bf.loc[ind2,'T_M1+Y'] + 4.16
bf.loc[ind2,'T_M1-Y'] = bf.loc[ind2,'T_M1-Y'] + 2.31

fig = plt.figure(figsize=[12,4])
plt.plot(bf['dts'],bf['T_M1+Y']-bf['T_M1-Y'],'.',markersize = 3,color='silver')
plt.plot(af['dts'],af['T_M1+Y']-af['T_M1-Y'],'.',markersize = 3,color='b')
plt.ylim(-2,2)
plt.xlabel('Date')
plt.ylabel(r'$\Delta{T}_{M}$ [$C^\circ$]')

plt.tick_params(axis='both',which = 'both',direction='in', width=2,length = 7)
plt.xticks([])

#plt.savefig('/Users/melisa/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/plots/SPIE_poster_figs/post_calibration.png')


Show seasonal variation and results of operation

In [ ]:
%matplotlib notebook

date = datetime.date(year = 2017, month = 8, day =3)

#  AO system requirements
cond1 = (env_data['dts'] > date) 
ind = np.where(cond1)[0]

cond2 = (env_data['dts'] < date) 
ind2 = np.where(cond2)[0]

af = cp.iloc[ind]
bf = cp.iloc[ind2]

#cp.loc[ind,'T_M1+Y'] = env_data.loc[ind,'T_M1+Y'] + 4.16
#cp.loc[ind,'T_M1-Y'] = env_data.loc[ind,'T_M1-Y'] + 2.31

fig = plt.figure(figsize=[16,9])
gs = gridspec.GridSpec(3, 1, height_ratios=[3,2,2]) 

#fig, (ax1,ax2) = plt.subplots(2,1, sharex = True, figsize=[12,6])

ax1 = plt.subplot(gs[0])
ax1.plot(cp['dts'],cp['T_M1+Y'],'.',markersize = 3,color='silver')
ax1.plot(af['dts'],af['T_M1+Y'],'.',markersize = 3,color='b')

#ax1.legend(['Before','After'],loc=3,facecolor='white',markerscale=6,frameon=False)

ax1.axvline(x=datetime.date(month=1,year = 2014, day =1),color = 'r',linewidth = 3,linestyle='--')
ax1.axvline(x=datetime.date(month=1,year = 2015, day =1),color = 'r',linewidth = 3,linestyle='--')
ax1.axvline(x=datetime.date(month=1,year = 2016, day =1),color = 'r',linewidth = 3,linestyle='--')
ax1.axvline(x=datetime.date(month=1,year = 2017, day =1),color = 'r',linewidth = 3,linestyle='--')
ax1.axvline(x=datetime.date(month=1,year = 2018, day =1),color = 'r',linewidth = 3,linestyle='--')
ax1.axvline(x=datetime.date(month=1,year = 2019, day =1),color = 'r',linewidth = 3,linestyle='--')



ax1.annotate('Year 1',[datetime.date(month=4,year = 2014, day =15),16],fontsize =20,color = 'r')
ax1.annotate('Year 2',[datetime.date(month=4,year = 2015, day =15),16],fontsize =20,color = 'r')
ax1.annotate('Year 3',[datetime.date(month=4,year = 2016, day =15),16],fontsize =20,color = 'r')
ax1.annotate('Year 4',[datetime.date(month=4,year = 2017, day =15),16],fontsize =20,color = 'r')
ax1.annotate('Year 5',[datetime.date(month=4,year = 2019, day =15),16],fontsize =20,color = 'r')



ax1.tick_params(axis='both',which = 'both',direction='in', width=2,length = 7, bottom = False, top=True,)
ax1.set_xticks([])
#ax1.set_xticks()
            
ax1.set_ylabel('M+Y Temperature [$C^\circ$]')

ax2 = plt.subplot(gs[1])
ax2.plot(cp['dts'],cp['T_M1+Y']-cp['T_M1-Y'],'.',markersize = 3,color='silver')
ax2.plot(af['dts'],af['T_M1+Y']-af['T_M1-Y'],'.',markersize = 3,color='b')

ax2.axvline(x=datetime.date(month=1,year = 2014, day =1),color = 'r',linewidth = 3,linestyle='--')
ax2.axvline(x=datetime.date(month=1,year = 2015, day =1),color = 'r',linewidth = 3,linestyle='--')
ax2.axvline(x=datetime.date(month=1,year = 2016, day =1),color = 'r',linewidth = 3,linestyle='--')
ax2.axvline(x=datetime.date(month=1,year = 2017, day =1),color = 'r',linewidth = 3,linestyle='--')
ax2.axvline(x=datetime.date(month=1,year = 2018, day =1),color = 'r',linewidth = 3,linestyle='--')
ax2.axvline(x=datetime.date(month=1,year = 2019, day =1),color = 'r',linewidth = 3,linestyle='--')



ax2.set_ylim(-5,2)
ax2.set_ylabel(r'$\Delta{T}_{M}$ [$C^\circ$]')

ax3 = plt.subplot(gs[2])
ax3.plot(bf.loc[ind2,'dts'],(bf.loc[ind2,'T_M1+Y'] + 4.16) - (bf.loc[ind2,'T_M1-Y'] + 2.31),'.',markersize = 3,color='silver')
ax3.plot(af['dts'],af['T_M1+Y']-af['T_M1-Y'],'.',markersize = 3,color='b')

ax3.axvline(x=datetime.date(month=1,year = 2014, day =1),color = 'r',linewidth = 3,linestyle='--')
ax3.axvline(x=datetime.date(month=1,year = 2015, day =1),color = 'r',linewidth = 3,linestyle='--')
ax3.axvline(x=datetime.date(month=1,year = 2016, day =1),color = 'r',linewidth = 3,linestyle='--')
ax3.axvline(x=datetime.date(month=1,year = 2017, day =1),color = 'r',linewidth = 3,linestyle='--')
ax3.axvline(x=datetime.date(month=1,year = 2018, day =1),color = 'r',linewidth = 3,linestyle='--')
ax3.axvline(x=datetime.date(month=1,year = 2019, day =1),color = 'r',linewidth = 3,linestyle='--')



ax3.tick_params(axis='both',which = 'both',direction='in', width=2,length = 7)
ax3.set_xticks([datetime.date(month=1,year = 2014, day =1),
            datetime.date(month=1,year = 2015, day =1),
            datetime.date(month=1,year = 2016, day =1),
            datetime.date(month=1,year = 2017, day =1),
            datetime.date(month=1,year = 2018, day =1),
            datetime.date(month=6,year = 2014, day =1),
            datetime.date(month=6,year = 2015, day =1),
            datetime.date(month=6,year = 2016, day =1),
            datetime.date(month=6,year = 2017, day =1),
            datetime.date(month=6,year = 2018, day =1)])


yearsFmt = mdates.DateFormatter('%Y/%m')
ax3.xaxis.set_major_formatter(yearsFmt)
ax3.set_ylim(-3,4)
ax3.set_xlabel('Date')
ax3.set_ylabel(r'$\Delta{T}_{M}$ [$C^\circ$]')
plt.tight_layout()

#plt.savefig('/Users/melisatallis/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/plots/SPIE_poster_figs/calibration_proc.png')


Show fit of bc data on af data 

One year at a time in between steps

In [ ]:
date = datetime.date(year = 2017, month = 8, day =2)
ind1 = np.where(env_data['dts'] <= date)[0]
ind2 = np.where(env_data['dts'] > date)[0]

bf = cp.iloc[ind1].sort_values(by='dts').set_index(np.arange(len(ind1)))
af = cp.iloc[ind2].sort_values(by = 'dts').set_index(np.arange(len(ind2)))
    
#  Replace year of dts to 2000
bf.loc[:,'dts'] = bf['dts'].apply(lambda dt: dt.replace(year = 2000 )) 
bf.loc[:,'MJDOBS'] = aptime.Time(bf['dts'].dt.to_pydatetime(), format='datetime').mjd  # Calculate MJD date after changing year 
bf = bf.sort_values(by ='dts').set_index(np.arange(len(bf))).dropna(subset=[sensor])  #  Sort the values in order by date

af.loc[:,'dts'] = af['dts'].apply(lambda dt: dt.replace(year = 2000 ))
af.loc[:,'MJDOBS'] = aptime.Time(af['dts'].dt.to_pydatetime(), format='datetime').mjd
af = af.sort_values(by ='dts').set_index(np.arange(len(af))).dropna(subset=[sensor])

All years combined steps

In [ ]:
#  Split data into before and after calibration data frames. 
date = datetime.date(year = 2017, month = 8, day =2)
ind1 = np.where(env_data['dts'] <= date)[0]
ind2 = np.where(env_data['dts'] > date)[0]

bf = env_data.iloc[ind1].sort_values(by='dts').set_index(np.arange(len(ind1)))
af = env_data.iloc[ind2].sort_values(by = 'dts').set_index(np.arange(len(ind2)))
    
bf_date1 = datetime.datetime(year = yr, month = 8, day =2) # lower bound of data beofre calibration
bf_date2 = datetime.datetime(year = yr, month = 12, day = 31)

cond1 = bf['dts'] > bf_date1
cond2  = bf['dts'] <= bf_date2
bf_ind = np.where(cond1 & cond2)[0]
        
# pick points that overlap with af dates
bf = bf.iloc[bf_ind].sort_values(by='dts').set_index(np.arange(len(bf_ind))).dropna(subset=[sensor])
    
#  Replace year of after to coincide with year of before
af.loc[:,'dts'] = af['dts'].apply(lambda dt: dt.replace(year = yr))
af.loc[:,'MJDOBS'] = aptime.Time(af['dts'].dt.to_pydatetime(), format='datetime').mjd
af = af.sort_values(by ='dts').set_index(np.arange(len(af))).dropna(subset=[sensor])

In [ ]:
%matplotlib notebook

plt.figure(1,figsize = [12,8])
plt.plot(bf['MJDOBS'],bf['T_M1+Y'],'.',markersize=2,color ='silver')
plt.plot(af['MJDOBS'],af['T_M1+Y'],'.',markersize=2,color='b')

def func(x, a, b):
    return a*x + b
    
date = datetime.date(year = 2000, month = 8, day =2)

#  Input arrays for curve fit
x = bf.loc[ind,'MJDOBS']
y = bf.loc[ind,sensor]
     
#  model calibration model
popt,pcov = optimize.curve_fit(func,x,y)

plt.plot(x,func(x))

#yearsFmt = mdates.DateFormatter('%Y/%m')
#plt.xaxis.set_major_formatter(yearsFmt)

plt.ylabel('Temperature [C]')
plt.xlabel('Date')
plt.xticks(rotation = 20)

#plt.savefig(savedir+'bf_cal'+dstr+'.png',dpi = 300)

In [ ]:
savedir = os.environ["HOME"]+'/Dropbox (GPI)/TEST_SCRATCH/scratch/mtallis/plots/calibration/'
dstr = time.strftime('%Y%m%d')